## Investigação Operacional
---

### TOC
- [Exercícios de Investigação Operacional](#1a): Problema 3 e Problema 5
- [O Caso Rio Negro](#1b)

In [3]:
using JuMP
using Cbc
using NamedArrays

<a id='1b'></a>
### Exercícios de Investigação Operacional
---

**Problema 3:**
Um produto em fabrico resulta duma montagem constituída por duas peças, A e B. Para a elaboração dessas peças recorre-se a uma máquina M1 e a cinco máquinas M2. A produtividade de cada máquina relativamente às duas peças é a indicada na tabela 1:

In [4]:
# Tempo de produção (em minutos por peça) das peças A e B nas máquinas M1 e M2
tabela1 = NamedArray([3 20; 5 15], ([:A, :B], [:M1, :M2]), (:Peça, :Máquina))

2×2 Named Array{Int64,2}
Peça ╲ Máquina │ :M1  :M2
───────────────┼─────────
:A             │   3   20
:B             │   5   15

A carga das máquinas M2 é repartida igualmente pelas 5 máquinas. O objectivo do problema é saber como se pode obter o máximo de montagens completas por dia. Considere que um dia corresponde a 8 horas de trabalho.


(a) Apresente um modelo matemático para este problema.

**R:** Iniciamos o modelo com 4 variáveis:

$x_{11}$ - peças A produzidas por M1

$x_{12}$ - peças B produzidas por M1

$x_{21}$ - peças A produzidas por M2

$x_{22}$ - peças B produzidas por M2

In [5]:
prob3 = Model(solver=CbcSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is CbcMathProg

In [6]:
@variable(prob3, x[1:2, 1:2] >=0)

2×2 Array{JuMP.Variable,2}:
 x[1,1]  x[1,2]
 x[2,1]  x[2,2]

In [7]:
horas = 8*60
@constraint(prob3, tempo_m1, sum(tabela1[:, :M1][l]*x[1, l] for l in 1:2) <= horas)

3 x[1,1] + 5 x[1,2] ≤ 480

In [8]:
horas = 8*60
@constraint(prob3, tempo_m2, sum(tabela1[:, :M2][l]*x[2, l] for l in 1:2) <= horas*5)

20 x[2,1] + 15 x[2,2] ≤ 2400

Como uma montgem completa é composta por uma peça do tipo A e outra do tipo B, nosso objetivo é:
        
$max (Z = min(x_{11} + x_{12}, x_{21} + x_{22}))$

Que vamos linearizar criando uma nova variável $Y = min$, e as restrições relativas à produção de M1 e M2 (que devem ser maiores que o mínimo):

$max Z = Y$

$\mbox{suj. a } x_{11} + x_{12} - Y > 0$

$\mbox{suj. a } x_{21} + x_{22} - Y > 0$

In [9]:
@variable(prob3, y >=0)

y

In [10]:
@objective(prob3, Max, y)

y

In [11]:
@constraint(prob3, min_m1, sum(x[1, l] for l in 1:2) - y >= 0)

x[1,1] + x[1,2] - y ≥ 0

In [12]:
@constraint(prob3, min_m2, sum(x[2, l] for l in 1:2) - y >= 0)

x[2,1] + x[2,2] - y ≥ 0

In [13]:
@time solve(prob3)

  2.713850 seconds (1.62 M allocations: 84.941 MiB, 1.16% gc time)


:Optimal

In [14]:
getvalue(x)

2×2 Array{Float64,2}:
 160.0    0.0
   0.0  160.0

In [15]:
getobjectivevalue(prob3)

160.0

(b) Considere agora a situação em que também se pretende manter uma utilização equilibrada entre as máquinas de modo que nenhuma delas seja utilizada mais 30 minutos por dia do que qualquer outra das máquinas.
Será possível resolver este novo problema por Programação Linear? Justifique.

**R:** Queremos que a diferença entre o tempo de A, $T_A = 3x_{1,1} + 5x_{1,2}$, e o tempo de cada máquina de B,  $T_B = \frac{20x_{2,1} + 15x{2,2}}{5}$, não exceda 30 minutos. Podemos linearizar esse problema da seguinte forma:

$|3x_{1,1} + 5x_{1,2} - (4x_{2,1} + 3x_{2,2})| \leq 30 \rightarrow 3x_{1,1} + 5x_{1,2} - (4x_{2,1} + 3x_{2,2}) \leq 30\mbox{ e }4x_{2,1} + 3x_{2,2} - (3x_{1,1} + 5x_{1,2}) \geq 30$

<a id='1b'></a>
---

**Problema 5:**

Uma companhia de navegação possui um navio com 3 porões de carga (à proa, à ré e ao centro) possuindo os limites de capacidade apresentados na tabela 1:

In [16]:
tabela1 = NamedArray([2000 100000; 3200 14000; 1800 80000], 
                    ([:Proa, :Centro, :Re], 
                    [:Tonelagem, :Volume]), (:Porao, :x))

3×2 Named Array{Int64,2}
Porao ╲ x │ :Tonelagem     :Volume
──────────┼───────────────────────
:Proa     │       2000      100000
:Centro   │       3200       14000
:Re       │       1800       80000

À empresa são oferecidas as cargas da tabela 2, cada uma das quais pode ser aceite parcial ou totalmente:

In [17]:
tabela2 = NamedArray([7000 60 20; 6500 50 24; 4000 25 16], 
                    ([:A, :B, :C], 
                    [:Peso, :Volume, :Lucro]), (:Carga, :x))

3×3 Named Array{Int64,2}
Carga ╲ x │   :Peso  :Volume   :Lucro
──────────┼──────────────────────────
:A        │    7000       60       20
:B        │    6500       50       24
:C        │    4000       25       16

A fim de preservar o equilíbrio do navio, deve manter-se a proporção entre o peso em cada porão e o volume respectivo. Admita que em cada porão podem ser transportadas partes de cargas diferentes. Pretende-se maximizar o lucro da empresa, relativo à utilização deste navio. Construa um modelo de LP para o problema apresentado.

**R:** Iniciamos o modelo com 6 variáveis:

$x_{i, j}$ - quantidade de toneladas de carga $i$ a ser transportada no porão $j$

In [18]:
prob5 = Model(solver=CbcSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is CbcMathProg

In [19]:
packs = [:Proa :Centro :Re]
segments = [:A :B :C]

n_packs = size(packs)[2] # 3
n_segments = size(segments)[2] # 3

3

In [20]:
@variable(prob5, x[1:n_packs, 1:n_segments] >=0)

3×3 Array{JuMP.Variable,2}:
 x[1,1]  x[1,2]  x[1,3]
 x[2,1]  x[2,2]  x[2,3]
 x[3,1]  x[3,2]  x[3,3]

In [21]:
@objective(prob5, Max, sum(tabela2[i, :Lucro] * sum(x[i, j] for j in 1:n_segments) for i in 1:n_packs))

20 x[1,1] + 20 x[1,2] + 20 x[1,3] + 24 x[2,1] + 24 x[2,2] + 24 x[2,3] + 16 x[3,1] + 16 x[3,2] + 16 x[3,3]

In [22]:
# Restrição de máximo de carga disponível
for i in 1:n_packs
    @constraint(prob5, sum(x[i,j] for j in 1:n_segments) <= tabela2[i, :Peso])
end

In [23]:
# Restrição de volume total em cada porão
for j in 1:n_segments
    @constraint(prob5, sum(x[i,j]*tabela2[i, :Volume] for i in 1:n_packs) <= tabela1[j, :Volume])
end

In [24]:
# Restrição de tonelagem total em cada porão
for j in 1:n_segments
    @constraint(prob5, sum(x[i,j] for i in 1:n_packs) <= tabela1[j, :Tonelagem])
end

Para manter a proporção de capacidade / volume no preenchimento dos porões, precisamos garantir, por exemplo:

$\frac{60x_{1,1} +􏰇 50x_{2,1} + 25x_{3,1}}{x_{1,1} +􏰇 x_{2,1} + x_{3,1}} = \frac{100000}{2000} = 50$

Esse é o caso para o porão `Proa`, indicado aqui como linha 1. Simplificando a expressão, ficamos com: $10x_{1,1} - 25x_{3, 1} = 0$.

Fazendo o mesmo processo para os outros porões, temos:

$178x_{1,2} + 146x_{2,2} + 66x_{3,2} = 0$

$100x_{1,3} + 72x_{2, 3} + 37x_{3,3} = 0$.

In [25]:
# Manutenção da proporção entre o volume e o peso total do porão
@constraint(prob5, razao1, 10*x[1,1] - 25*x[3,1] == 0)
@constraint(prob5, razao2, 178*x[1,2] + 146*x[2,2] + 66*x[3,2] == 0)
@constraint(prob5, razao3, 100*x[1,3] + 72*x[2,3] + 37*x[3,3] == 0)

100 x[1,3] + 72 x[2,3] + 37 x[3,3] = 0

In [26]:
prob5

Maximization problem with:
 * 12 linear constraints
 * 9 variables
Solver is CbcMathProg

In [27]:
@time solve(prob5)

  0.001283 seconds (79 allocations: 8.375 KiB)


:Optimal

In [28]:
getvalue(x)

3×3 Array{Float64,2}:
    0.0  0.0  0.0
 2000.0  0.0  0.0
    0.0  0.0  0.0

In [29]:
getobjectivevalue(prob5)

48000.0

<a id='1b'></a>
### O Caso Rio Negro
---

Reprodução do caso descrito em *História e Metodologia da Investigação Operacional* (CARRAVILHA, M. A. - FEUP, 2001).

Cada sessão nesse arquivo (e seu respectivo texto) é apresentada na apostila de referência. As sessões foram reordenadas de acordo com a lógica de formulação dos modelos.

### TOC

1. [Análise da situação da fábrica](#1)

2. [Análise econômica da empresa](#2)
3. [Quantificação da poluição](#3)

Primeiro modelo:
4. [Decisão do município: Limite de poluição](#4)

Segundo modelo:
5. [E se se construísse um sistema de tratamento das águas residuais?](#5)
6. [Decisão do município: Sistema de tratamento + limite de poluição](#6)

Solução adotada:
7. [Solução do Eng. Luís Bela Vida](#7)

<a id='1'></a>
### Análise da situação da fábrica
O complexo agro-industrial tem, basicamente, duas linhas de produtos, A e B, costumando produzir, por mês, 20 toneladas de A e 80 toneladas de B, respeitando-se assim a sua capacidade máxima total de produção mensal, que é de 100 toneladas/mês. O seu director diz, com orgulho, que há mais de 10 anos que adoptam esta solução.

In [2]:
modelo = Model(solver=CbcSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is CbcMathProg

In [3]:
L = 2 # linhas de producao: A e B

2

In [4]:
@variable(modelo, x[1:L] >=0)

2-element Array{JuMP.Variable,1}:
 x[1]
 x[2]

In [5]:
max_producao = 100
@constraint(modelo, producao, sum(x[l] for l = 1:L) <= max_producao)

x[1] + x[2] ≤ 100

<a id=2></a>
### Análise econômica da empresa
Identificação dos lucros relativos às duas linhas de produção (difícil dado o deficiente sistema contabilístico da Superterra):

- Existe uma despesa de 280000 contos/mês mesmo que não haja produção;
- Sem contar com esse encargo, o lucro obtido por tonelada de A e B é de 2000 e 4000 contos, respectivamente;
- Dada a sólida posição da empresa no mercado, as actuais produções, ou mesmo produções superiores, são facilmente escoadas para o mercado.

In [6]:
lucro = [2000 4000]
despesa = 280000

280000

In [7]:
@objective(modelo, Max, sum(lucro[l]*x[l] for l in 1:L) - despesa)

2000 x[1] + 4000 x[2] - 280000

In [8]:
# Lucro atual: 20t A, 80t B
20*lucro[1] + 80*lucro[2] - 280000

80000

<a id=3></a>
### Quantificação da poluição
Indicador da presença de matéria orgânica — CBO5

- Durante o Verão a capacidade de recepção do rio não ultrapassa os 210 mg/l de CBO5 no caudal de águas residuais.
- A análise do sistema de produção permitiu concluir que cada tonelada produzida de A é responsável por uma carga de 0.7 mg/l de CBO5, e cada tonelada de B, por 3.5 mg/l.
- O caudal residual não depende das quantidades produzidas de cada tipo de produto.

In [9]:
cbo5 = [0.7 3.5]

1×2 Array{Float64,2}:
 0.7  3.5

<a id=4></a>
### Decisão do município: Limite de poluição

**Obrigar a empresa a respeitar o limite de 210 mg/l** ⇒ redução das produções, pois: 

20 × 0.7 + 80 × 3.5 = 294 > 210

Para tal sugeriu que a empresa produzisse na mesma proporção de 1 para 4 (produtos A e B) mas em quantidades tais que o limite de 210 mg/l fosse respeitado, isto é:

0.7x + 3.5 × 4x = 210 ⇔ x = 14.3 ⇔ A → 14.3 ton/mês ∧ B → 57.2 ton/mês 

O que daria um lucro de:
L = 2000 × 14.3 + 4000 × 57.2 − 280000 = −22600 contos/mês

A empresa rejeitou esta solução pois daria prejuízo. 

*Vamos modelar abaixo a solução ótima para esse cenário, sem a limitaç˜:*

In [10]:
max_cbo5 = 210
@constraint(modelo, cbo5, sum(x[l]*cbo5[l] for l = 1:L) <= max_cbo5)

0.7 x[1] + 3.5 x[2] ≤ 210

In [11]:
modelo

Maximization problem with:
 * 2 linear constraints
 * 2 variables
Solver is CbcMathProg

In [12]:
@time solve(modelo, relaxation=true) # relaxation=True permite valores contínuos

  2.426727 seconds (1.65 M allocations: 85.788 MiB, 1.37% gc time)


:Optimal

In [13]:
getvalue(x)

2-element Array{Float64,1}:
 50.0
 50.0

In [14]:
getobjectivevalue(modelo)

20000.00000000006

\# [para plotar, precisa instalar pkg](https://stackoverflow.com/questions/39586694/plot-linear-equations-inequalities-in-julia)
```
using Pkg
Pkg.add("ImplicitEquations")
using ImplicitEquations, Plots

plot(cbo5, 1:100, 1:100)
```

<a id=5></a>
### E se se construísse um sistema de tratamento das águas residuais?
Atendendo aos problemas de poluição já apresentados, a empresa encomendou um projecto de construção de um sistema de tratamento das suas águas residuais, tendo-se concluído que as cargas passavam a 0.6 e 3.0 mg/l, para A e B, respectivamente. Todavia, os custos fixos aumentariam de 20000 contos, e o lucro unitário reduzir-se-ia de 20%, o que foi considerado uma exorbitância.

In [15]:
sistema = [0.6 3.0]

1×2 Array{Float64,2}:
 0.6  3.0

In [16]:
# Despesa aumenta em 20000
despesa2 = despesa + 20000

300000

In [17]:
# Lucro unitário reduz 20%
lucro2 = lucro * 0.8

1×2 Array{Float64,2}:
 1600.0  3200.0

<a id=6></a>
### Decisão do município: Sistema de tratamento + limite de poluição

**No caso da introdução do sistema de tratamento:**

0.6 × x + 3.0 × 4x = 210 ⇔ x = 16.7 e portanto: L = 1600 × 16.7 + 3200 × 66.8 − 300000 = −59520 contos/mês

O que também foi considerado inaceitável pois dava um prejuízo maior!

*Vamos modelar abaixo a solução ótima para esse cenário:*

OBS: como ainda [não é possível mudar coeficientes de restrições](http://www.juliaopt.org/JuMP.jl/v0.12/probmod.html#modifying-constraints), vamos iniciar outro modelo

In [18]:
modelo2 = Model(solver=CbcSolver())
@variable(modelo2, x[1:L] >=0, Int)

2-element Array{JuMP.Variable,1}:
 x[1]
 x[2]

In [19]:
max_producao = 100
@constraint(modelo2, producao, sum(x[l] for l = 1:L) <= max_producao)

x[1] + x[2] ≤ 100

In [20]:
@constraint(modelo2, cbo5, sum(x[l]*sistema[l] for l = 1:L) <= max_cbo5)

0.6 x[1] + 3 x[2] ≤ 210

In [21]:
@objective(modelo2, Max, sum(lucro2[l]*x[l] for l in 1:L) - despesa2)

1600 x[1] + 3200 x[2] - 300000

In [22]:
@time solve(modelo2, relaxation=true)

  0.000859 seconds (81 allocations: 5.141 KiB)


:Optimal

In [23]:
getvalue(x)

2-element Array{Float64,1}:
 37.5
 62.5

In [24]:
getobjectivevalue(modelo2)

-40000.00000000003

<a id=7></a>
*Solução do Eng. Luís Bela Vida*: Repartir os custos da necessária alteração de produção de (20,80) para (50,50) pela SuperTerra e pelos operadores turísticos.

- Lucro actual: 2×20+4×80−280 = 80 ($10^{3}$ contos/mês)
- Lucro após alteração: 2 × 50 + 4 × 50 − 280 = 20 ($10^{3}$ contos/mês) 

In [27]:
atual = 20*lucro[1] + 80*lucro[2] - 280000

80000

In [26]:
apos_alteracao = getobjectivevalue(modelo)

20000.00000000006

Considerando os 3 meses de Verão o prejuízo por ano será de 180000 contos (por sinal muito inferior ao resultante da introdução da estação de tratamento...).

In [28]:
12* atual - (9*atual + 3*apos_alteracao)

179999.99999999977